In [ ]:
import torch
from torch.utils.data import DataLoader
import cellshape_cloud as cscld
from cellshape_cluster.deep_embedded_clustering import DeepEmbeddedClustering
from cellshape_cluster.training_functions import train
from chamferdist import ChamferDistance

In [ ]:
autoencoder = cscld.GraphAutoEncoder(
    num_features=128, k=20, encoder_type="dgcnn"
)


model = DeepEmbeddedClustering(autoencoder, 10)

In [ ]:
from torch import nn

PATH_TO_DATASET = "/home/mvries/Documents/Datasets/ModelNet10Voxel/Train"
batch_size = 16
learning_rate = 0.00001


dataset = cscld.PointCloudDataset(PATH_TO_DATASET)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
dataloader_inf = DataLoader(dataset, batch_size=1, shuffle=False)

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate * 16 / batch_size,
    betas=(0.9, 0.999),
    weight_decay=1e-6,
)

recon_crit = ChamferDistance()
clust_crit = nn.KLDivLoss(reduction="sum")

train(
    model,
    dataloader,
    dataloader_inf,
    1,
    optimizer,
    recon_crit,
    clust_crit,
    1,
    0.01,
    2,
    "./",
)